In [3]:
import pandas as pd

In [4]:
fips = pd.read_csv('raw/mt-county-codes.csv', dtype={'number': str})

In [5]:
fips.head()

,geography,name,region,fips,number
0,"Beaverhead County, Montana",Beaverhead,Mountains,30001,18
1,"Big Horn County, Montana",Big Horn,Plains,30003,22
2,"Blaine County, Montana",Blaine,Plains,30005,24
3,"Broadwater County, Montana",Broadwater,Mountains,30007,43
4,"Carbon County, Montana",Carbon,Plains,30009,10


In [13]:
file = 'raw/city-county-prop-tax-analysis.xlsx'
dtype = {
    'County #': str
}
# Note: read_excel() pulls from Excel files like read_csv() pulls from .csv's
raw_tax = pd.read_excel(file, dtype=dtype, sheet_name='TaxesByCo')
raw_total_res_tax = pd.read_excel(file, dtype=dtype, sheet_name='Class4ResTaxesByCo')
raw_tax_per_capita = pd.read_excel(file, dtype=dtype, sheet_name='County Per Capita Total Taxes')
raw_res_tax_per_capita = pd.read_excel(file, dtype=dtype, sheet_name='County Per Capita Res Taxes')
raw_res_per_income = pd.read_excel(file, dtype=dtype, sheet_name='County Res Per Income')

In [7]:
# Preview data
raw_res_per_income.head()

,County #,County,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,$ Change,% Change
0,18,Beaverhead,0.017072,0.016347,0.016468,0.015878,0.016658,0.015424,0.015598,0.015991,...,0.015860,0.015206,0.014763,0.013704,0.013779,0.013907,0.015472,0.015420,-0.001652,-0.096763
1,22,Big Horn,0.006593,0.005495,0.005164,0.004888,0.004948,0.004337,0.004512,0.004159,...,0.004354,0.004521,0.004712,0.005096,0.005639,0.005492,0.006294,0.006152,-0.000441,-0.066920
2,24,Blaine,0.009071,0.009021,0.008250,0.008089,0.008857,0.008010,0.008216,0.007332,...,0.006678,0.006933,0.006553,0.006601,0.008369,0.008375,0.010174,0.009610,0.000539,0.059401
3,43,Broadwater,0.012547,0.013940,0.013273,0.013298,0.013985,0.013075,0.012588,0.013435,...,0.013474,0.015355,0.013325,0.014696,0.014485,0.013888,0.015494,0.014863,0.002317,0.184657
4,10,Carbon,0.020474,0.021542,0.019766,0.020118,0.019960,0.021517,0.022047,0.023747,...,0.021797,0.020998,0.021747,0.021534,0.019901,0.019732,0.022059,0.020472,-0.000002,-0.000104


In [23]:
# Numn counties w/ per-cap taxes rising faster than inflation(40% 02 --> 2018)
w_inflation = raw_res_tax_per_capita.copy()
w_inflation['p_change'] = w_inflation[2002] / w_inflation[2018]
len(w_inflation[w_inflation['p_change'] >= .4])

52

In [14]:
# Drop total line at end by trimming to 56 counties
raw_tax = raw_tax[0:56]
raw_total_res_tax = raw_total_res_tax[0:56]
raw_tax_per_capita = raw_tax_per_capita[0:56]
raw_res_tax_per_capita = raw_res_tax_per_capita[0:56]
raw_res_per_income = raw_res_per_income[0:56]

# Drop $ change and % change columns - we'll do those calculations ourselves
raw_res_tax_per_capita.drop(
    labels=['$ Change', '% Change'], # names of columns to drop
    axis=1, # specifies drop based on columns instead of rows (axis=0)
    inplace=True, # modify existing dataframe in place instead of returning new one
)
raw_res_per_income.drop(
    labels=['$ Change', '% Change'], 
    axis=1,
    inplace=True,
)

In [15]:
df_res_per_capita = raw_res_tax_per_capita.melt(
    id_vars=['County #', 'County'],
    var_name='year',
    value_name='residential_taxes_per_capita'
)
df_total_per_capita = raw_tax_per_capita.melt(
    id_vars=['County #', 'County'],
    var_name='year',
    value_name='total_taxes_per_capita'
)
df_total = raw_tax.melt(
    id_vars=['County #', 'County'],
    var_name='year',
    value_name='total_taxes'
)
df_residential = raw_total_res_tax.melt(
    id_vars=['County #', 'County'],
    var_name='year',
    value_name='residential_taxes'
)
df_res_per_income = raw_res_per_income.melt(
    id_vars=['County #', 'County'],
    var_name='year',
    value_name='res_per_income'
)

In [12]:
df_res_per_income.head()

,County #,County,year,res_per_income
0,18,Beaverhead,2002,0.017072
1,22,Big Horn,2002,0.006593
2,24,Blaine,2002,0.009071
3,43,Broadwater,2002,0.012547
4,10,Carbon,2002,0.020474


In [41]:
# df_res.to_csv('data/per-capita-residential-taxes-by-county.csv', index=False)

In [16]:
# Merging
combined = df_total.merge(df_residential[['County','year','residential_taxes']], on=['County','year'])
combined = combined.merge(df_total_per_capita[['County','year','total_taxes_per_capita']], on=['County', 'year'])
combined = combined.merge(df_res_per_capita[['County','year','residential_taxes_per_capita']], on=['County','year'])
combined = combined.merge(df_res_per_income[['County','year','res_per_income']], on=['County','year'])

# Add FIPS code column
combined = combined.merge(fips[['fips','number','region']], left_on='County #', right_on='number').drop('number', axis=1)

# calculating new data fields
combined['res_percent'] = combined['residential_taxes'] / combined['total_taxes']
combined['non_res_taxes'] = combined['total_taxes'] - combined['residential_taxes']
combined['non_res_taxes_per_capita'] = combined['total_taxes_per_capita'] - combined['residential_taxes_per_capita']

In [17]:
print(len(combined), 'rows')
combined.head()

952 rows


,County #,County,year,total_taxes,residential_taxes,total_taxes_per_capita,residential_taxes_per_capita,res_per_income,fips,region,res_percent,non_res_taxes,non_res_taxes_per_capita
0,18,Beaverhead,2002,7.991143e+06,3.512553e+06,886.132476,389.504703,0.017072,30001,Mountains,0.439556,4.478589e+06,496.627774
1,18,Beaverhead,2003,7.937825e+06,3.547598e+06,889.491834,397.534541,0.016347,30001,Mountains,0.446923,4.390227e+06,491.957293
2,18,Beaverhead,2004,8.405847e+06,3.779371e+06,943.628935,424.267076,0.016468,30001,Mountains,0.449612,4.626475e+06,519.361858
3,18,Beaverhead,2005,8.631169e+06,3.824307e+06,969.358630,429.504347,0.015878,30001,Mountains,0.443081,4.806863e+06,539.854283
4,18,Beaverhead,2006,8.974881e+06,4.029238e+06,995.881139,447.096995,0.016658,30001,Mountains,0.448946,4.945643e+06,548.784144


In [16]:
combined.to_csv('data/taxes-by-county.csv', index=False)
# combined.to_csv('data/taxes-by-county.csv', index=False)
# index=False option keeps row index numbers at the lefthand side of the dataframe from cluttering the data file

In [157]:
df = combined.copy()
base = combined[combined['year'] == 2002].copy()
base['baseline'] = base['res_per_income']
df = combined.merge(base[['County','baseline']], on='County', how='left')
df['index_res_taxes_per_income'] = df['res_per_income'] / df['baseline'] * 100
df = df.drop(labels='baseline', axis=1)
df

,County #,County,year,total_taxes,residential_taxes,per_capita_taxes,per_capita_residential_taxes,res_per_income,fips,res_percent,index_res_taxes_per_income
0,18,Beaverhead,2002,7.991143e+06,3.512553e+06,886.132476,389.504703,0.017072,30001,0.439556,100.000000
1,18,Beaverhead,2003,7.937825e+06,3.547598e+06,889.491834,397.534541,0.016347,30001,0.446923,95.749588
2,18,Beaverhead,2004,8.405847e+06,3.779371e+06,943.628935,424.267076,0.016468,30001,0.449612,96.459892
3,18,Beaverhead,2005,8.631169e+06,3.824307e+06,969.358630,429.504347,0.015878,30001,0.443081,93.005859
4,18,Beaverhead,2006,8.974881e+06,4.029238e+06,995.881139,447.096995,0.016658,30001,0.448946,97.569628
...,...,...,...,...,...,...,...,...,...,...,...
947,3,Yellowstone,2014,1.877598e+08,8.360138e+07,1209.208111,538.408470,0.011472,30111,0.445257,82.698317
948,3,Yellowstone,2015,2.056161e+08,9.387609e+07,1313.555492,599.716907,0.012457,30111,0.456560,89.796982
949,3,Yellowstone,2016,2.096900e+08,9.516113e+07,1327.967466,602.655599,0.012628,30111,0.453818,91.031452
950,3,Yellowstone,2017,2.290827e+08,1.070937e+08,1438.699539,672.576325,0.013479,30111,0.467489,97.163239
